In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Configuration
START_DATE = '2022-06-07'
END_DATE = '2024-06-07'
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

def generate_load_profile(date, hour):
    """Generate synthetic load value for given datetime and hour"""
    # Base daily pattern (evening peak at ~6:30 PM)
    if date.weekday() < 5:  # Weekdays
        base_pattern = [
            0.12,  # 12-1 AM (step 1)
            0.10,  # 1-2 AM (step 2)
            0.08,  # 2-3 AM (step 3)
            0.07,  # 3-4 AM (step 4)
            0.07,  # 4-5 AM (step 5)
            0.15,  # 5-6 AM (step 6)
            0.25,  # 6-7 AM (step 7)
            0.40,  # 7-8 AM (step 8)
            0.55,  # 8-9 AM (step 9)
            0.60,  # 9-10 AM (step 10)
            0.65,  # 10-11 AM (step 11)
            0.60,  # 11 AM-12 PM (step 12)
            0.55,  # 12-1 PM (step 13)
            0.50,  # 1-2 PM (step 14)
            0.55,  # 2-3 PM (step 15)
            0.65,  # 3-4 PM (step 16)
            0.75,  # 4-5 PM (step 17)
            0.85,  # 5-6 PM (step 18)
            0.95,  # 6-7 PM (step 19) PEAK HOUR
            0.85,  # 7-8 PM (step 20)
            0.70,  # 8-9 PM (step 21)
            0.50,  # 9-10 PM (step 22)
            0.30,  # 10-11 PM (step 23)
            0.20   # 11 PM-12 AM (step 24)
        ]
    else:  # Weekends (Friday & Saturday)
        base_pattern = [
            0.15,  # 12-1 AM
            0.12,  # 1-2 AM
            0.10,  # 2-3 AM
            0.09,  # 3-4 AM
            0.09,  # 4-5 AM
            0.20,  # 5-6 AM
            0.30,  # 6-7 AM
            0.45,  # 7-8 AM
            0.60,  # 8-9 AM
            0.70,  # 9-10 AM
            0.75,  # 10-11 AM
            0.80,  # 11 AM-12 PM
            0.75,  # 12-1 PM
            0.70,  # 1-2 PM
            0.75,  # 2-3 PM
            0.80,  # 3-4 PM
            0.85,  # 4-5 PM
            0.90,  # 5-6 PM
            0.92,  # 6-7 PM (slightly lower peak)
            0.85,  # 7-8 PM
            0.75,  # 8-9 PM
            0.60,  # 9-10 PM
            0.40,  # 10-11 PM
            0.25   # 11 PM-12 AM
        ]
    
    # Seasonal adjustment (winter = Nov-Feb)
    month = date.month
    if month in [11, 12, 1, 2]:  # Winter
        multiplier = 1.25
    elif month in [6, 7, 8]:  # Summer
        multiplier = 0.85
    else:  # Shoulder seasons
        multiplier = 1.0
    
    # Get base value
    value = base_pattern[hour]
    
    # Apply seasonal multiplier
    value *= multiplier
    
    # Add random fluctuation (±8%)
    value *= np.random.uniform(0.92, 1.08)
    
    # Ensure value < 1 and reasonable minimum
    return min(max(value, 0.05), 0.99)

# Generate date range
dates = pd.date_range(start=START_DATE, end=END_DATE)

# Create data
data = []
for date in dates:
    for hour in range(24):
        value = generate_load_profile(date, hour)
        data.append({
            'date': date.strftime('%Y-%m-%d'),
            'step': hour + 1,
            'value': round(value, 6)
        })

# Create DataFrame and save
df = pd.DataFrame(data)


In [6]:
df = df[:17545]

In [7]:
df

,date,step,value
0,2022-06-07,1,0.099952
1,2022-06-07,2,0.091130
2,2022-06-07,3,0.070524
3,2022-06-07,4,0.060439
4,2022-06-07,5,0.056225
...,...,...,...
17540,2024-06-06,21,0.596775
17541,2024-06-06,22,0.443978
17542,2024-06-06,23,0.257533
17543,2024-06-06,24,0.183423


In [9]:
df['step'] = range(1,17546)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9448\2783929921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['step'] = range(1,17546)


In [10]:
df

,date,step,value
0,2022-06-07,1,0.099952
1,2022-06-07,2,0.091130
2,2022-06-07,3,0.070524
3,2022-06-07,4,0.060439
4,2022-06-07,5,0.056225
...,...,...,...
17540,2024-06-06,17541,0.596775
17541,2024-06-06,17542,0.443978
17542,2024-06-06,17543,0.257533
17543,2024-06-06,17544,0.183423


In [11]:
df.to_csv('morocco_residential_load.csv', index=False)
print(f"Generated CSV with {len(df)} rows ({(len(df)/24):.0f} days)")

Generated CSV with 17545 rows (731 days)


# Check for difference in shape

In [22]:
wind = pd.read_csv(r"C:\Users\LENOVO\Desktop\data_for_simulation\wind_meknes.csv",index_col=0)

In [23]:
wind

,time,step,wspd
0,6/7/2022 0:00,1,3.6
1,6/7/2022 1:00,2,7.6
2,6/7/2022 2:00,3,7.6
3,6/7/2022 3:00,4,3.6
4,6/7/2022 4:00,5,7.6
...,...,...,...
17540,6/6/2024 20:00,17541,20.5
17541,6/6/2024 21:00,17542,14.8
17542,6/6/2024 22:00,17543,11.2
17543,6/6/2024 23:00,17544,11.2


In [20]:
cons = pd.read_csv(r"C:\Users\LENOVO\Desktop\data_for_simulation\morocco_residential_load.csv")

In [21]:
cons

,date,step,value
0,2022-06-07,1,0.099952
1,2022-06-07,2,0.091130
2,2022-06-07,3,0.070524
3,2022-06-07,4,0.060439
4,2022-06-07,5,0.056225
...,...,...,...
17540,2024-06-06,17541,0.596775
17541,2024-06-06,17542,0.443978
17542,2024-06-06,17543,0.257533
17543,2024-06-06,17544,0.183423
